In [6]:
import os
import pandas as pd

# Path to the directory containing the CSV files
directory_path = '/data1-8tb-blk/sp2023stock/Insight-Wave/TopicModeling/result/integrationOutput/id/thansettakij_ids.csv'

articles_list = {}

df = pd.read_csv(directory_path)
articles_list = df['ID'].tolist()

print(f'max = {max(articles_list)}, min ={min(articles_list)}')


max = 584734, min =463653


In [7]:
import os
main = os.getcwd()
dataset_thansettakij = main + '/../../../data/thansettakij/article/'

start_process = main +  '/../../../src/Parsing/thansettakij/parsing_progress.txt'
end_process = main + '/../../../src/Crawling/thansettakij/progress.txt'

In [8]:
import os
import csv
import re
import pandas as pd
import chardet
from datetime import datetime
from bs4 import BeautifulSoup

# Detect input file
def htmlInput(file_path):
    with open(file_path, "r", encoding='utf-8') as file:
        html_content = file.read()
    soup = BeautifulSoup(html_content, "html.parser")
    return soup

#Extract title
def extract_title(soup):
    title = soup.title.string.strip() if soup.title else None
    return title

#Extract body
def extract_body(soup):
    article_divs = soup.find_all('div', class_="detail")
    articles = []

    if article_divs:
        for article_div in article_divs:
            content_details = article_div.find_all('div', class_='content-detail')
            for content_detail in content_details:
                p_tags = content_detail.find_all('p')
                if not p_tags:
                    parts = content_detail.stripped_strings
                    for part in parts:
                        articles.append(part)
                else:
                    for p_tag in p_tags:
                        articles.append(p_tag.get_text(strip=True))

    articles = [s.replace('\x8d', ' ') for s in articles]
    articles = [item for item in articles if item != '']

    # Remove the caption text
    articles = [re.sub(r'\[caption.*?\[/caption\]', '', s) for s in articles]

    return articles

#Extract tags
def extract_tags(soup):
    tag_div = soup.find('div', class_='cat--title')
    tags = []

    if tag_div:
        tag_content = tag_div.get_text(strip=True)
    else: 
        tag_content = None
    return tag_content
def extract_pubdate(soup):
    month_list = {
        'มกราคม': '01', 
        'กุมภาพันธ์': '02', 
        'มีนาคม': '03', 
        'เมษายน': '04', 
        'พฤษภาคม': '05', 
        'มิถุนายน': '06', 
        'กรกฎาคม': '07', 
        'สิงหาคม': '08', 
        'กันยายน': '09', 
        'ตุลาคม': '10', 
        'พฤศจิกายน': '11', 
        'ธันวาคม': '12'
    }

    date_time_div = soup.find('div', class_='date')
    pubdate = None

    if date_time_div is not None:
        date_time_string = date_time_div.get_text(strip=True)

        # Split the string into words
        words = date_time_string.split()

        # Identify the components
        day = words[0]
        month = month_list.get(words[1])  # Get the numeric month
        year = words[2]
        if year.isdigit():
            year = str(int(year) - 543)

        # Form the date and time in the desired format
        pubdate = f"{year}-{month}-{day} "

    return pubdate

#Extract title
def extract_intro(soup):
    intro_div = soup.find("h2", class_="content-blurb")
    intro = ""
    if intro_div:
        intro = intro_div.get_text(strip=True)
    else:
        intro = None

    return intro

def extract_urlPic(soup):
    meta_tag = soup.find('meta', attrs={"property": "og:image"})
    if meta_tag:
        return meta_tag.get('content')
    else:
        print("No meta tag with property 'og:image' found.")
    return None

# #Start
# if os.path.exists(start_process):
#     with open('parsing_progress.txt','r') as f:
#         progress_start = int(f.readline().strip())
#         start_id = progress_start - 1
# else: 
#     print("Start path doesn't exists process start from 1")
#     start_id = 1 
    
# article_id = start_id

# #End
# if os.path.exists(end_process):
#     with open(process,'r') as f:
#         progress_end = int(f.readline().strip())
#         end_id = progress_end + 1
# else: 
#     print("End path doesn't exists")
# print(f'Start parsing at {start_id} and Ending at {end_id}' )


print(f'Start parsing process')
for article_id in articles_list: #range(start_id, end_id+1):
    file_path = dataset_thansettakij + str(article_id) + "/index.txt"
    print(f'Processing article id {article_id}')

    if os.path.exists(file_path):
        try:
        
            soup = htmlInput(file_path)
            title = extract_title(soup)
            intro = extract_intro(soup)
            articles = extract_body(soup)
            pubdate = extract_pubdate(soup)
            tag_content = extract_tags(soup)
            url = extract_urlPic(soup)

            if title == intro:
                intro = None
            if articles and intro == articles[0]:
                articles.pop(0)
            if title == articles[0]:
                articles.pop(0)


            data_dict = {'Title': title, 'Intro': intro, 'Article': articles, 'DateTime': pubdate, 'Tags': tag_content, 'url_picture': url}
            
            output_path = dataset_thansettakij+ str(article_id) + "/parsing.txt"
            with open(os.path.join(output_path), 'w', encoding="utf-8") as f:
                for key, value in data_dict.items():
                    if value is not None:
                        if key == 'Tags' and isinstance(value, list):
                            f.write(f"[::{key}::]\n")
                            for tags in value:
                                f.write(f"{tags}\n")

                        elif key == 'Article' and isinstance(value, list):
                            f.write(f"[::{key}::]\n")
                            for bodys in value:
                                if bodys == 'SPONSORED':
                                    continue
                                else:
                                    f.write(f"{bodys}\n")
                                    f.write("\n")
                        else:
                            f.write(f"[::{key}::]\n{value}\n")
                            
                            
            with open('parsing_progress.txt', 'w') as f:
                f.write(f'{article_id}')

            with open('parsing_number.csv', 'a', newline='') as f:
                writer = csv.writer(f)
                validation = 'Valid'
                writer.writerow([article_id, validation])

                    
        except Exception as e:
            print(f"Error processing article {article_id}: {e}")
            with open('parsing_number.csv', 'a', newline='') as f:
                writer = csv.writer(f)
                validation = 'Error'
                writer.writerow([article_id, validation])
                continue


# Write the last processed article_id to 'csv_progress.txt' after loop ends
with open('parsing_progress.txt', 'w') as f:
    f.write(str(article_id) + '\n')

Start parsing process
Processing article id 584734
Processing article id 584732
Processing article id 584729
Processing article id 584728
Processing article id 584727
Processing article id 584726
Processing article id 584725
Processing article id 584722
Processing article id 584721
Processing article id 584720
Processing article id 584718
Processing article id 584716
Processing article id 584715
Processing article id 584714
Processing article id 584712
Processing article id 584710
Processing article id 584709
Processing article id 584707
Processing article id 584703
Processing article id 584701
Processing article id 584700
Processing article id 584699
Processing article id 584697
Processing article id 584696
Processing article id 584695
Processing article id 584693
Processing article id 584692
Processing article id 584691
Processing article id 584690
Processing article id 584689
Processing article id 584688
Processing article id 584687
Processing article id 584685
Processing article id